In [307]:
import functools
import requests
import re
from urllib.parse import urlparse



# Only need this until the raw data get fixed
def fix_twitter(twitter_url):
    if twitter_url == "https://witter.com/f4f_wuerzburg":
        return "https://twitter.com/f4f_wuerzburg"
    if twitter_url == "https://twitter.com/fridaysforfutureBrüggen":
        return "https://twitter.com/brueggen4future"
    if twitter_url == "http://www.twitter.de/klimastreiklandau/":
        return "https://twitter.com/klimastreikld"
    if twitter_url == "https://twitter.com/FDatteln?s=09":
        return None
    if twitter_url == "https://twitter.com/duisburgfff":
        return "https://twitter.com/fff_duisburg"
    if twitter_url == "https://twitter.com/fff_2206":
        return "https://twitter.com/fff_2209"
    if twitter_url == "https://twitter.com/F4F_Potsdam":
        return None
    if twitter_url == "https://twitter.com/ForTubingen":
        return None
    else:
        return twitter_url
    
raw_data = list(map(lambda entry: dict(entry, twitter=fix_twitter(entry["twitter"])), requests.get('https://api.fffutu.re/v1/localgroups').json()))

In [308]:
data = filter(lambda x: x["twitter"] is not None, raw_data)

def url_to_handle(url): 
    match = re.match("https?://(www\.|mobile\.)?twitter.com/([a-zA-Z_0-9]*)", url)
    if match is not None:
        return match.group(2);
    else:
        # Follow redirects
        redirect_url = requests.get(url).url
        if redirect_url != url:
            return url_to_handle(redirect_url)
        else:
            return url;
data = list(map(lambda data: dict(data, twitter_handle=url_to_handle(data["twitter"])), data))
print(data)

[{'id': 2985, 'name': 'Aachen', 'lat': 50.776351, 'lon': 6.083862, 'state': 'Nordrhein-Westfalen', 'facebook': 'https://www.facebook.com/FridaysForFutureAC/', 'instagram': 'https://www.instagram.com/fridaysforfuture.ac/', 'twitter': 'https://twitter.com/FutureFridaysAC', 'youtube': None, 'website': 'https://fridaysforfuture.de/ortsgruppen/aachen/', 'whatsapp': 'https://chat.whatsapp.com/KNXj3DZd4GCEo9OB2Z46tE', 'telegram': 'https://t.me/FFF_AC_Info', 'other': None, 'email': 'aachen@fridaysforfuture.is', 'twitter_handle': 'FutureFridaysAC'}, {'id': 2705, 'name': 'Amberg', 'lat': 49.4454211, 'lon': 11.858728, 'state': 'Bayern', 'facebook': 'https://facebook.com/events/2484756088462386/?ti=as', 'instagram': 'https://www.instagram.com/p/B5R9gvjK-v1/?igshid=ukntx4er7g9d', 'twitter': 'https://twitter.com/fffamberg', 'youtube': None, 'website': None, 'whatsapp': 'https://chat.whatsapp.com/FGe6ZuoXyc73aiG3hmiDSp', 'telegram': None, 'other': None, 'email': 'amberg@fridaysforfuture.is', 'twitter

In [309]:
import json

twitter_auth = None
with open("twitter_token") as f:
    twitter_auth = json.load(f)
auth_header = dict(authorization=f"Bearer {twitter_auth['bearer']}")

In [310]:
requests.get("https://api.twitter.com/2/users/by?usernames=fridayforfuture&user.fields=public_metrics", headers=auth_header).json()

{'data': [{'username': 'FridayForFuture',
   'public_metrics': {'followers_count': 142997,
    'following_count': 774,
    'tweet_count': 2205,
    'listed_count': 520},
   'name': 'Fridays For Future Germany',
   'id': '1072187272815149057'}]}

In [311]:
len(data)

134

In [312]:
def chunks(lst, chunk_size):
    """ Yields successive chunks from the list."""
    for i in range(0, len(lst), chunk_size):
        yield lst[i:i + chunk_size]

def get_metrics_bulk(user_names):
    if len(user_names) > 100:
        collected_results = map(get_metrics_bulk, chunks(user_names, 100))
        return {username: stats for result in collected_results for username, stats in result.items()}
    user_names_param = functools.reduce(lambda a, b: a + ',' + b, user_names)
    api_url = f"https://api.twitter.com/2/users/by?usernames={user_names_param}&user.fields=public_metrics"
    raw_response = requests.get(api_url, headers=auth_header).json()
    return dict(map(lambda user: (user["username"].lower(), user['public_metrics']), raw_response['data']))

twitter_metrics = get_metrics_bulk(list(map(lambda entry: entry["twitter_handle"], data)))
print(twitter_metrics)

{'futurefridaysac': {'followers_count': 3646, 'following_count': 232, 'tweet_count': 1005, 'listed_count': 48}, 'fffamberg': {'followers_count': 222, 'following_count': 110, 'tweet_count': 172, 'listed_count': 7}, 'fffannabergb': {'followers_count': 264, 'following_count': 40, 'tweet_count': 86, 'listed_count': 10}, 'fff_augsburg': {'followers_count': 1604, 'following_count': 210, 'tweet_count': 248, 'listed_count': 28}, 'aurichforfuture': {'followers_count': 349, 'following_count': 88, 'tweet_count': 527, 'listed_count': 10}, 'fridaysforbk': {'followers_count': 298, 'following_count': 39, 'tweet_count': 120, 'listed_count': 12}, 'fridaysforfutu8': {'followers_count': 54, 'following_count': 66, 'tweet_count': 31, 'listed_count': 7}, 'fbadtoelz': {'followers_count': 377, 'following_count': 5, 'tweet_count': 311, 'listed_count': 11}, 'fffbbg_': {'followers_count': 694, 'following_count': 57, 'tweet_count': 469, 'listed_count': 26}, 'fff_berlin': {'followers_count': 22742, 'following_coun

In [313]:
list(map(lambda og: dict(og, twitter_metrics=twitter_metrics[og["twitter_handle"].lower()]), data))

[{'id': 2985,
  'name': 'Aachen',
  'lat': 50.776351,
  'lon': 6.083862,
  'state': 'Nordrhein-Westfalen',
  'facebook': 'https://www.facebook.com/FridaysForFutureAC/',
  'instagram': 'https://www.instagram.com/fridaysforfuture.ac/',
  'twitter': 'https://twitter.com/FutureFridaysAC',
  'youtube': None,
  'website': 'https://fridaysforfuture.de/ortsgruppen/aachen/',
  'whatsapp': 'https://chat.whatsapp.com/KNXj3DZd4GCEo9OB2Z46tE',
  'telegram': 'https://t.me/FFF_AC_Info',
  'other': None,
  'email': 'aachen@fridaysforfuture.is',
  'twitter_handle': 'FutureFridaysAC',
  'twitter_metrics': {'followers_count': 3646,
   'following_count': 232,
   'tweet_count': 1005,
   'listed_count': 48}},
 {'id': 2705,
  'name': 'Amberg',
  'lat': 49.4454211,
  'lon': 11.858728,
  'state': 'Bayern',
  'facebook': 'https://facebook.com/events/2484756088462386/?ti=as',
  'instagram': 'https://www.instagram.com/p/B5R9gvjK-v1/?igshid=ukntx4er7g9d',
  'twitter': 'https://twitter.com/fffamberg',
  'youtube': 